# Comparison: GAPoT vs IEEE 1459 vs p-q Theory

This notebook demonstrates the comparison between Geometric Algebra Power Theory (GAPoT),
IEEE Standard 1459, and Instantaneous Power (p-q) Theory for a circuit with harmonic distortion.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Import GAPoT package
from gapot import GeometricPower, FourierToGA
from gapot import IEEE1459Power, PQTheory
from gapot.utils import generate_distorted_signal, create_comparison_table

plt.style.use('seaborn-v0_8-whitegrid')
print("Packages loaded successfully!")

## 1. Generate Test Signals

Create voltage and current signals with fundamental and harmonic components:
- Fundamental: 230V, 10A, power factor 0.866 (30° lagging)
- 5th harmonic: 5% voltage THD, 20% current THD
- 7th harmonic: 3% voltage THD, 14% current THD

In [ ]:
# Signal parameters
f1 = 50.0       # Fundamental frequency (Hz)
fs = 50000.0    # Sampling frequency (Hz)
cycles = 2      # Number of cycles

# Fundamental parameters
U1 = 230.0      # RMS voltage (V)
I1 = 10.0       # RMS current (A)
phi1 = np.pi/6  # 30° lagging (PF = 0.866)

# Harmonics: (h, U_h/U1, I_h/I1, phase_shift)
harmonics = [
    (5, 0.05, 0.20, 0.3),   # 5th harmonic
    (7, 0.03, 0.14, 0.2),   # 7th harmonic
]

# Generate signals
u, i, t = generate_distorted_signal(
    U1=U1, I1=I1, phi1=phi1,
    harmonics=harmonics,
    f1=f1, fs=fs, cycles=cycles
)

print(f"Generated {len(u)} samples over {cycles} cycles")
print(f"Sampling frequency: {fs} Hz")

In [ ]:
# Plot signals
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

t_ms = t * 1000  # Convert to milliseconds

axes[0].plot(t_ms, u, 'b-', linewidth=1)
axes[0].set_ylabel('Voltage (V)')
axes[0].set_title('Distorted Voltage Waveform')
axes[0].grid(True, alpha=0.3)

axes[1].plot(t_ms, i, 'r-', linewidth=1)
axes[1].set_ylabel('Current (A)')
axes[1].set_xlabel('Time (ms)')
axes[1].set_title('Distorted Current Waveform')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. GAPoT Analysis

Calculate geometric power using the GAPoT framework.

In [ ]:
# Create GeometricPower object
gapot = GeometricPower(u, i, f1=f1, fs=fs)

# Print summary
print(gapot.summary())

## 3. IEEE 1459 Analysis

Calculate power according to IEEE Standard 1459-2010.

In [ ]:
# Create IEEE1459 object
ieee = IEEE1459Power(u, i, f1=f1, fs=fs)

# Print summary
print(ieee.summary())

## 4. p-q Theory Analysis

Calculate instantaneous power using Akagi's p-q theory.

In [ ]:
# Create PQTheory object
pq = PQTheory(u, i, fs=fs)

# Print summary
print(pq.summary())

## 5. Comparison Table

In [ ]:
# Create comparison table
print(create_comparison_table(u, i, f1, fs))

In [ ]:
# Visual comparison
methods = ['GAPoT', 'IEEE 1459', 'p-q Theory']
P_values = [gapot.P, ieee.P, pq.p_avg]
Q_values = [gapot.M_Q_norm, ieee.Q1, abs(pq.q_avg)]

x = np.arange(len(methods))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
bars1 = ax.bar(x - width/2, P_values, width, label='Active Power (W)', color='steelblue')
bars2 = ax.bar(x + width/2, Q_values, width, label='Reactive Power (var)', color='coral')

ax.set_ylabel('Power')
ax.set_title('Power Comparison Across Methods')
ax.set_xticks(x)
ax.set_xticklabels(methods)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar in bars1 + bars2:
    height = bar.get_height()
    ax.annotate(f'{height:.1f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),
                textcoords="offset points",
                ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## 6. GAPoT Unique Capability: Cross-Frequency Analysis

GAPoT can explicitly identify cross-frequency interactions that other methods aggregate into a single distortion term.

In [ ]:
# Show cross-frequency distortion terms
print("Cross-frequency distortion components (M_D):")
print("-" * 50)
if gapot.M_D_components:
    for (f_m, f_n), D_mn in sorted(gapot.M_D_components.items()):
        h_m = int(f_m / f1)
        h_n = int(f_n / f1)
        print(f"  h={h_m} × h={h_n} ({f_m:.0f}Hz × {f_n:.0f}Hz): D = {D_mn:.4f} va")
else:
    print("  No significant cross-frequency terms detected.")

print(f"\nTotal ||M_D|| = {gapot.M_D_norm:.4f} va")
print(f"IEEE 1459 D = {ieee.D:.4f} va (aggregated)")

## 7. Energy Conservation Verification

Verify that GAPoT satisfies energy conservation: ||M||² = S² = U_rms² × I_rms²

In [ ]:
# Verify energy conservation
verification = gapot.verify_energy_conservation()

print("Energy Conservation Verification")
print("=" * 50)
print(f"S_geometric (||M||)    = {verification['S_geometric']:.6f} VA")
print(f"S_traditional (U×I)    = {verification['S_traditional']:.6f} VA")
print(f"Relative error         = {verification['relative_error']*100:.6f}%")
print()
print(f"P² + ||M_Q||² + ||M_D||² = {verification['P_squared_plus_nonactive']:.4f}")
print(f"||M||²                   = {verification['S_squared']:.4f}")

## 8. Conclusions

Key observations from this comparison:

1. **Active power (P)**: All methods agree on the active power calculation.

2. **Reactive power**: GAPoT's M_Q and IEEE 1459's Q1 represent fundamental reactive power, while p-q theory's q includes all frequencies.

3. **Distortion power**: GAPoT uniquely provides explicit cross-frequency terms, while IEEE 1459 aggregates everything into D.

4. **Energy conservation**: GAPoT satisfies ||M||² = S², consistent with traditional apparent power.

5. **Geometric insight**: GAPoT reveals the structure of power flow through bivector decomposition.